# Worksheet 08

Name:  Naima Abrar
UID: U04989541

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.116571839127025, 3.498876332011147, 5.574479013440579, 8.392027512234645, 4.502680660050708, 4.980143962090419, 2.870666097177706, 6.56473094983, 3.712224727220725, 6.221817586986688]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
m2 = 8
std2 = 1
s2 = np.random.normal(m2, std2, 10).tolist()
print(s2)

[6.892493354496817, 7.949210115518857, 7.544017350458421, 7.888586505523621, 8.050807827390765, 7.56020300319763, 9.03279244505825, 6.488964962138527, 9.004882261362539, 6.536588691003165]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
c1 = s1
c2 = s2
data = []
for i in range(10):
    flip = random.choice([0, 1])
    if flip == 0 and c1!=[]:
        p1 = c1.pop()
        data.append(p1)
    elif flip == 1 and c2!=[]:
        p2 = c2.pop()
        data.append(p2)
print(data)

[6.221817586986688, 3.712224727220725, 6.56473094983, 6.536588691003165, 9.004882261362539, 2.870666097177706, 4.980143962090419, 4.502680660050708, 6.488964962138527, 9.03279244505825]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Means 5,8
number of components -> 2
Data Points

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [7]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [len(s1)/(len(s1)+len(s2)),len(s2)/(len(s1)+len(s2))]
mean = [sum(s1)/len(s1), sum(s2)/len(s2)]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1),sum(map(lambda x: (x - mean[1])**2, s2)) / len(s2)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[6.8705160389184226, 7.804479932406963, 8.577020727889815, 7.561302831685851, 6.946107988533101]
[5.494464399514581, 4.711346114107617, 4.303523597296517, 5.517542247358652, 4.911852887446327]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 7.551885503886831,  mean_2 = 4.987745849144739
var_1 = 0.3892051209546817,  var_2 = 0.21755291158750975


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


the estimated probabilities do match the true values. The estimated variane, though, is a bit lower than the actual variance

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [8]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append((pdf_i[0]*prob_s[0])/prob_x)
    prob_s1_x.append((pdf_i[1]*prob_s[1])/prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

#the first 5 points have a high probbability of belinging to cluster 2
#points with valuea above 6 belong to the other cluster


point =  5.494464399514581
probability of observing that point if it came from cluster 0 =  8.765143063039603e-07
probability of observing that point if it came from cluster 1 =  0.12170554294348689
point =  4.711346114107617
probability of observing that point if it came from cluster 0 =  2.7817253404720784e-12
probability of observing that point if it came from cluster 1 =  0.818155195068371
point =  4.303523597296517
probability of observing that point if it came from cluster 0 =  7.667676020240167e-16
probability of observing that point if it came from cluster 1 =  0.013044312695383673
point =  5.517542247358652
probability of observing that point if it came from cluster 0 =  1.1970795129255922e-06
probability of observing that point if it came from cluster 1 =  0.0945284784162988
point =  4.911852887446327
probability of observing that point if it came from cluster 0 =  1.0461017728546129e-10
probability of observing that point if it came from cluster 1 =  1.7255176417429263
point

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [9]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x),sum(prob_s1_x)/len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0]*x[1] for x in zip(prob_s1_x,data)])/sum(prob_s1_x) ]
var = [sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)])/sum(prob_s0_x),sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 7.551877388033618,  mean_2 = 4.987743777454323
var_1 = 0.3892201533886599,  var_2 = 0.21755269521373272


teh updates values are very close to the values we started out with and this indicates that the clsutering was done quite accurately

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [11]:
#the probabilities are more accurate
#the general assignments to the clusters largely remains the same


point =  5.494464399514581
probability of observing that point if it came from cluster 0 =  0.002781225965265126
probability of observing that point if it came from cluster 1 =  0.4740679321720034
point =  4.711346114107617
probability of observing that point if it came from cluster 0 =  2.015130424504125e-05
probability of observing that point if it came from cluster 1 =  0.7175869673662717
point =  4.303523597296517
probability of observing that point if it came from cluster 0 =  8.297096876494775e-07
probability of observing that point if it came from cluster 1 =  0.2916373104053787
point =  5.517542247358652
probability of observing that point if it came from cluster 0 =  0.00313991859904908
probability of observing that point if it came from cluster 1 =  0.44870874123406396
point =  4.911852887446327
probability of observing that point if it came from cluster 0 =  8.267549502242265e-05
probability of observing that point if it came from cluster 1 =  0.8440713093182769
point =  6.8

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [12]:
for p0, p1, point in zip(new_prob_s0_x, new_prob_s1_x, data):
    cluster = 0 if p0 > p1 else 1
    print(f"Point {point} is assigned to cluster {cluster}")


Point 5.494464399514581 is assigned to cluster 1
Point 4.711346114107617 is assigned to cluster 1
Point 4.303523597296517 is assigned to cluster 1
Point 5.517542247358652 is assigned to cluster 1
Point 4.911852887446327 is assigned to cluster 1
Point 6.8705160389184226 is assigned to cluster 0
Point 7.804479932406963 is assigned to cluster 0
Point 8.577020727889815 is assigned to cluster 0
Point 7.561302831685851 is assigned to cluster 0
Point 6.946107988533101 is assigned to cluster 0
